In [1]:
import numpy as np
import pandas as pd
import json

import gurobipy as gb
from gurobipy import GRB

In [2]:
c = pd.read_json('./cleandata/dist_mat.json')
t = pd.read_json('./cleandata/time_mat.json')
locdem = pd.read_excel('./cleandata/locdem.xlsx')
#8 - 124,11 - 125
c = c.loc[:123, :123].to_numpy()
t = t.loc[:123, :123].to_numpy()
q = locdem['Number of pallets']
q = q.to_numpy()

def get(i):
    if i == 124:
        return 8
    if i == 125:
        return 11
    return i

def time(i,j):
    i,j = get(i), get(j)
    return t[i,j]

def cost(i,j):
    if (i,j) in [(125,11), (11,125), (8,124), (124,8)]:
        return 1e+8
    i,j = get(i), get(j)
    return c[i,j]

In [3]:
v = 126
V = set(range(126))
def Del(p):
    return V - set([p])
N = Del(0)
Q = 9

from itertools import product
A = filter(lambda tup: tup[0] != tup[1], product(N,N))

In [4]:
m1 = gb.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12


In [5]:
#Remember to take care of 124,125
#8 - 124,11 - 125

In [6]:
x = m1.addVars(v,v,vtype=GRB.BINARY, name='x')
m1.setObjective(sum([cost(i,j) * x[i,j] for i,j in product(V,V)]), GRB.MINIMIZE)

In [7]:
#Source and sink constraints
for i in N:
    m1.addConstr(sum([x[i,j] for j in Del(i)]) == 1)
for j in N:
    m1.addConstr(sum([x[i,j] for i in Del(j)]) == 1 )

In [8]:
#Construction of L loops
L = m1.addVar(lb=0.0, vtype=GRB.INTEGER, name='L')
m1.addConstr(sum([x[0,j] for j in Del(0)]) == L)


<gurobi.Constr *Awaiting Model Update*>

In [9]:
#Capacity constraints
u = m1.addVars(V, vtype=GRB.INTEGER, name='u')
for i,j in A:
    m1.addConstr(u[i] - u[j] + Q*x[i,j] <= Q - q[j])
for i in V:
    m1.addConstr(u[i] >= q[i])

In [10]:
# This doesn't work out: m1.optimize()